# DSCI 525 - Web and Cloud Computing
## Milestone 1: Tackling big data on your laptop

### Group #4
### Members: Heidi Ye, Junting He, Kamal MoravejJahromi, Tanmay Sharma

### GitHub Repo: **https://github.com/UBC-MDS/group4-525**

## Loading the libraries

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd
import pyarrow.feather as feather
import pyarrow.dataset as ds

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

#### Note: Code across this lab has been adapted from the DSCI-525 lectures

## 1. Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

## 2. Unzipping Data

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.7 s, sys: 2.73 s, total: 5.43 s
Wall time: 1min 27s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17.7 s, sys: 3.32 s, total: 21.1 s
Wall time: 21.5 s


## 3. Combining data CSVs

In [7]:
df = pd.read_csv("./figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv")
df

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [8]:
%%time
%memit

#Merging all the csv files

files = glob.glob('figsharerainfall/*NSW.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'/([^_]*)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 374.74 MiB, increment: 0.06 MiB
CPU times: user 6min 44s, sys: 23.1 s, total: 7min 7s
Wall time: 7min 15s


The following table summerizes the `cpu times` and `Wall time` of combining csv files for all team members.

|   |  Memory Usage |  CPU Time  | Wall Time  | OS  |  Memory (RAM) |  CPUs   |
|---|---|---|---|---|---|---|
|  Heidi | 0.04 MiB  | 6min 45s  | 7min 10s  |  mac OS Catalina | 16 GB 3733 MHz LPDDR4X | 2 GHz Quad-Core Intel Core i5   |
| Junting  | 0.05 MiB  |  15min 8s | 15min 36s  | mac OS Catalina  |16GB of 2400MHz DDR4 | 2.6GHz 6-core Intel Core i7   |
| Kamal  |  0.79 MiB |  9min 20s  |  9min 43s | Windows 10 Pro  | 8 GB DDR4 SDRAM |  Intel Core i7-8650U Quad-Core  |
|  Tanmay | 0.04 MiB  | 5min 23s  |  5min 28s |  mac OS Big Sur   | 16 GB 2667 MHz DDR4  | 2.6 GHz 6-Core Intel Core i7    |

- We were able to combine the CSV files using Pandas concat method on both macOS and Windows operating systems and on the machines of all the 4 team-members.
- `Memory usage` ranged from `0.04 MiB` to `0.79 MiB`, `CPU time` from `5min 23s` to `15min 8s` and Wall time from `5min 28s` to `15min 36s`. 
- Memory usage and CPU processing times are also impacted by background processes on the individual machines.

In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


## 4. Load the combined CSV to memory and perform a simple EDA

In [10]:
%%time
%%memit
#loading the entire data to the memory using Pandas
df = pd.read_csv("figsharerainfall/combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-ESM2           3541230
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
GFDL-CM4            3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 8403.29 MiB, increment: 4625.34 MiB
CPU times: user 1min 6s, sys: 13.3 s, total: 1min 19s
Wall time: 1min 22s




|   |  Memory Usage |  CPU Time  | Wall Time  | OS  |  Memory (RAM) |  CPUs   |
|---|---|---|---|---|---|---|
|  Heidi | 2933.94 MiB  | 54.9 s  | 1min 20s  |  mac OS Catalina | 16 GB 3733 MHz LPDDR4X | 2 GHz Quad-Core Intel Core i5   |
| Junting  | 4149.87 MiB  |  2min 54s | 3min 11s  | mac OS Catalina  |16GB of 2400MHz DDR4 | 2.6GHz 6-core Intel Core i7   |
| Kamal  |  1084.17 MiB |  4min 20s  |  4min 46s | Windows 10 Pro  | 8 GB DDR4 SDRAM |  Intel Core i7-8650U Quad-Core  |
|  Tanmay | 3921.75 MiB  | 1min 1s  | 1min 4s  | mac OS Big Sur   | 16 GB 2667 MHz DDR4  | 2.6 GHz 6-Core Intel Core i7 |

- We were able to load the combined CSV files using Pandas read_csv method on both macOS and Windows operating systems and on the machines of all the 4 team-members.
- `Memory usage` ranged from `1084.17 MiB` to `4149.87 MiB`, `CPU time` from `54.9 s` to `4min 20s` and Wall time from `1min 4s` to `4min 46s`. 
- Memory usage and CPU processing times are also impacted by background processes on the individual machines.

In [11]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [12]:
#checking datatypes for columns
df.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

### 4.1. Investigate approaches to reduce memory usage while performing the EDA 

### 4.1.1. Changing dtype of the data and loading just the columns we want

In [13]:
print(f"Memory usage with float64: {df[['lat_min','lat_max','rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max','rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 1499.23 MB
Memory usage with float32: 749.61 MB


### 4.1.2. Loading data in chunks using Pandas

In [14]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 6374.36 MiB, increment: 1354.38 MiB
CPU times: user 1min 6s, sys: 7.75 s, total: 1min 14s
Wall time: 1min 15s



|   |  Memory Usage |  CPU Time  | Wall Time  | OS  |  Memory (RAM) |  CPUs   |
|---|---|---|---|---|---|---|
|  Heidi | 5967.77 MiB | 53 s  | 1min 2s |  mac OS Catalina | 16 GB 3733 MHz LPDDR4X | 2 GHz Quad-Core Intel Core i5   |
| Junting  |2163.42  | 2min 18s | 2min 19s | mac OS Catalina  |16GB of 2400MHz DDR4 | 2.6GHz 6-core Intel Core i7   |
| Kamal  |  1619.86 MiB |  1min 54s  |  1min 59s | Windows 10 Pro  | 8 GB DDR4 SDRAM |  Intel Core i7-8650U Quad-Core  |
|  Tanmay |  1271.02 MiB | 55.6 s  |  56.5 s | mac OS Big Sur   | 16 GB 2667 MHz DDR4  | 2.6 GHz 6-Core Intel Core i7  |

- We were able to load data in chunks using Pandas on both macOS and Windows operating systems and on the machines of all the 4 team-members..
- `Memory usage` ranged from `1271.02 MiB` to `5967.77 MiB`, `CPU time` from `53 s` to `2min 18s` and Wall time from `56.5 s` to `2min 19s`. 
- Memory usage and CPU processing times are also impacted by background processes on the individual machines.

### 4.1.2. Loading data using Dask

In [15]:
%%time
%%memit
# Using dask
ddf = dd.read_csv('figsharerainfall/combined_data.csv')
print(ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 6610.67 MiB, increment: 2087.40 MiB
CPU times: user 1min 33s, sys: 24 s, total: 1min 57s
Wall time: 46 s




|   |  Memory Usage |  CPU Time  | Wall Time  | OS  |  Memory (RAM) |  CPUs   |
|---|---|---|---|---|---|---|
|  Heidi | 4714.38 MiB  | 1min 32s  | 42.6 s  |  mac OS Catalina | 16 GB 3733 MHz LPDDR4X | 2 GHz Quad-Core Intel Core i5   |
| Junting  | 1817.48 MiB  | 3min 43s | 1min 19s  | mac OS Catalina  |16GB of 2400MHz DDR4 | 2.6GHz 6-core Intel Core i7   |
| Kamal  |  1690.57 MiB|  2min 24s  |  1min 2s | Windows 10 Pro  | 8 GB DDR4 SDRAM |  Intel Core i7-8650U Quad-Core  |
|  Tanmay |  1797.21 MiB | 1min 31s  |  34.3 s | mac OS Big Sur   | 16 GB 2667 MHz DDR4  | 2.6 GHz 6-Core Intel Core i7   |

- We were able to load data using Dask on both macOS and Windows operating systems and on the machines of all the 4 team-members..
- `Memory usage` ranged from `1690.57 MiB` to `4714.38 MiB`, `CPU time` from `1min 31s` to `3min 43s` and Wall time from `34.3 s` to `1min 19s`. 
- Memory usage and CPU processing times are also impacted by background processes on the individual machines.

### 4.2. Discuss your observations.

We tried the following approaches to reduce the memory usage while performing the EDA:

1. Changing dtype of the data and loading just the columns we want:
    - We loaded only 3 out of the 5 attributes, namely, 'lat_min','lat_max', and 'rain (mm/day)'.
    - We changed the data type of these attributes to float32 from the orignial float64.
    - Memory usage with float32: 750.17 MB was almost half of the memory usage with float64: 1500.33 MB.
    - This validates the hypothesis that using lower data types(float32 vs 64 in this case) leads to more efficient memory usage.
    
2. Loading data in chunks using Pandas
    - We loaded the combined csv file using a chunksize=10_000_000 while performing the EDA.
    - We observed a decline in peak memory usage from 8403.29 MiB to 6374.36 MiB.
    - Wall Time decreased from 1min 22s to 1min 15s.
    - We do not see a significant change in the Wall and CPU times. 
    - We hypothesize that this impact would be more pronoucned when doing more memory intensive operations in EDA and using smaller chunk sizes would further reduce the memory usage.
    
3. Loading data using Dask
    - We next loaded the combined CSV using a dask object. 
    - We observed a decline in peak memory usage from 8403.29 MiB to 6610.67 MiB.
    - Wall Time decreased significantly from 1min 22s to 46 s.
    - We also notice that the CPU time was higher than the wall time (1min 33s vs 46s) suggesting that the CPU was performing operations in parallel. 

In summary, loading the entire data (combined_csv) to memory at once has the longest wall time and the highest memory usage as expected. We have looked at three different approaches to load the data more efficiently i.e. loading the entire data using pandas, loading the data in chunks, and loading data using Dask. We conclude that if we want to reduce the memory usage and the processing time, loading with Dask is the best option.

### 5. Perform a simple EDA in R

In [16]:
%%time
%%memit

dataset = ds.dataset("figsharerainfall/combined_data.csv", format="csv")

table = dataset.to_table()

peak memory: 5711.06 MiB, increment: 1216.03 MiB
CPU times: user 21.2 s, sys: 11.3 s, total: 32.5 s
Wall time: 29.9 s


In [17]:
%%time
# writing in feather format 
feather.write_feather(table, 'figsharerainfall/combined_data.feather')

CPU times: user 5.11 s, sys: 12.5 s, total: 17.6 s
Wall time: 6.92 s


In [20]:
%%time
%%R

library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figsharerainfall/combined_data.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 15.75686 secs
CPU times: user 11.9 s, sys: 22.6 s, total: 34.5 s
Wall time: 16.9 s


In [21]:
%%R

library(tidyverse)

r_table <- r_table %>% rename(rain_mmperday = `rain (mm/day)`)

summary_table <- r_table %>%
    drop_na() %>%
    summarise(median_lat_min = median(lat_min),
             median_lat_max = median(lat_max),
             median_lon_min = median(lon_min),
             median_lon_max = median(lon_max),
             median_rain = median(rain_mmperday))
    
summary_table


R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ stringr 1.4.0
✔ tidyr   1.1.3     ✔ forcats 0.5.1
✔ readr   1.4.0     

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# A tibble: 1 x 5
  median_lat_min median_lat_max median_lon_min median_lon_max median_rain
           <dbl>          <dbl>          <dbl>          <dbl>       <dbl>
1            -33          -32.0           147.           148.      0.0615


### 5.1 Discuss why you chose this approach over others

### 5.1.1. Reasons to choose feather

We chose `feather file format` based on the following reasons:

- The `feather` file format is **faster** compared with the `parquet` file and `arrow exchange` while writing files. It writes data with lesser serialization and deserialization that would result in a higher input/output speed. As we can see in our case, the Wall time for writing the feather file is almost half of the parquet file's wall time. 
- The `feather` file format can effectively **transfer between python and R programming languages** due to the embedded API that would result in faster reading and writing data using R.
- `Feather` also takes **fewer memories** compared with the CSV file format. We can see that the CSV file takes `5.7 GB` while the feather file format takes `1.1 GB` space. 

- We observed that the `partitioned.parquet` and `parquet files` take less space than the `feather` file format. However, the higher speed of writing and reading feather speeds the data queries and analysis.

- `Arrow` only support some operations. The `feather` does not have this limitation.

In summary, feather was selected over Parquet, Pandas Exchange, and Arrow Exchange for its comparatively high I/O speed, minimal memory on disk, and the fact that unpacking is not necessary for the data to be loaded back into RAM. Additionally, feather is relatively easy to use and is a suitable choice since the intent is not term storage. 

### 5.1.2. Challenges and discussion

One of biggest challenges associated with this size of data was that computational speed became extremely slow. It was not uncommon for simple tasks to take upwards of 15 minutes. In addition, even after the data was read in, the manipulation of data was still fairly slow. One approach was to read in the data in chunks to minimize the amount of data available at one time. Although this approach may work in some use cases, it is not without its limitations. For example, there may be instances where we need full access to all the data and chunking could result in sampling the data incorrectly. Other alternatives explored in this milestone include changing the dtype of the data as well as loading in data via Dask. Again, this provided some computational savings but likely would not scale well for even larger datasets. To tackle the insufficient memory challenge certain team members ended up deleting certain files and terminating applications to make more memory available. 

Another challenge was that runtime did vary from machine to machine. There were instances where the same code could take three or four times longer to run depending on the system being used. This type of inconsistency makes working in this environment fairly unpredictable under tight deadlines. In this milestone, there was no apparent method of overcoming this issue. 